In [1]:
import tensorflow as tf
import numpy as np
import cv2

import dlib
import pickle

2022-07-23 23:07:05.768493: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-23 23:07:05.768517: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
def get_model():

    backbone = tf.keras.applications.EfficientNetB2(
        input_shape = (96,96,3),
        include_top = False,
        weights = None,
    )

    model = tf.keras.Sequential([
        backbone,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(7, activation="softmax"),
    ])

    return model

In [3]:
model = get_model()
model.load_weights("best_weights.h5")

2022-07-23 23:07:07.722370: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-07-23 23:07:07.722411: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jyotsna-Khushixd): /proc/driver/nvidia/version does not exist
2022-07-23 23:07:07.722659: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
# load label encoder
def load_object(name):
    pickle_obj = open(f"{name}.pck", "rb")
    obj = pickle.load(pickle_obj)
    return obj

Le = load_object("LabelEncoder")

/home/jyotsna/anaconda3/lib/python3.9/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator LabelEncoder from version 1.0.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [10]:
def ProcessImage(image):
    image = tf.convert_to_tensor(image)
    image = tf.image.resize(image, [96,96], method="bilinear")
    image = tf.expand_dims(image,0)
    return image

def RealTimePrediction(image, model, encoder_):
    prediction = model.predict(image)
    prediction = np.argmax(prediction, axis=1)
    return encoder_.inverse_transform(prediction)[0]

def rect_to_bb(rect):
    x = rect.left()
    y = rect.top()
    w = rect.right() - x
    h = rect.bottom() - y
    return (x,y,w,h)

In [12]:
VideoCapture = cv2.VideoCapture(0)

detector = dlib.get_frontal_face_detector()

while True:
    ret, frame = VideoCapture.read()
    if not ret:
        break
        
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    rects = detector(gray, 0)
    
    if len(rects)>=1:
        for rect in rects:
            (x,y,w,h) = rect_to_bb(rect)
            img = gray[y-10 : y+h+10, x-10 : x+w+10]
            
            if img.shape[0]==0 or img.shape[1]==0:
                cv2.imshow("Frame", frame)
                
            else:
                img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
                img = ProcessImage(img)
                out = RealTimePrediction(img, model, Le)
                cv2.rectangle(frame, (x,y), (x+w, y+h), (0,255,0), 2)
                z = y-15 if (y-15)>15 else (y+15)
                cv2.putText(frame, str(out), (x,z), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,255,0), 2)
                
            cv2.imshow("Frame", frame)
            
        else:
            cv2.imshow("Frame", frame)
            
        if cv2.waitKey(1) & 0xFF == ord('q'):
                break
                
VideoCapture.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 38ms/step


QObject::moveToThread: Current thread (0x557b1db59170) is not the object's thread (0x557b1d9e1800).
Cannot move to target thread (0x557b1db59170)

QObject::moveToThread: Current thread (0x557b1db59170) is not the object's thread (0x557b1d9e1800).
Cannot move to target thread (0x557b1db59170)

QObject::moveToThread: Current thread (0x557b1db59170) is not the object's thread (0x557b1d9e1800).
Cannot move to target thread (0x557b1db59170)

QObject::moveToThread: Current thread (0x557b1db59170) is not the object's thread (0x557b1d9e1800).
Cannot move to target thread (0x557b1db59170)

QObject::moveToThread: Current thread (0x557b1db59170) is not the object's thread (0x557b1d9e1800).
Cannot move to target thread (0x557b1db59170)

QObject::moveToThread: Current thread (0x557b1db59170) is not the object's thread (0x557b1d9e1800).
Cannot move to target thread (0x557b1db59170)

QObject::moveToThread: Current thread (0x557b1db59170) is not the object's thread (0x557b1d9e1800).
Cannot move to tar

1/1 [==============================] - 0s 32ms/step


1/1 [==============================] - 0s 33ms/step


1/1 [==============================] - 0s 31ms/step
